In [4]:
nb_gNB_RX = 2
nb_UE_ports = 1


wireless_data_path = rf"CLICKTIME\data"
time_data_path = rf"CLICKTIME\time"


output_raw_path = "FALL-set"

In [5]:
import numpy as np 
import os
import scipy
from math import ceil
from scipy import linalg
import cmath

def generate_data_split_tags(nb_gNB_RX,nb_UE_ports):
    """
    根据设定的数据保存形式，我们将其拆分去除无关的标识符部分以及换行符，仅保留和数据相关的部分。
    由于标识符仅仅和天线数以及UE端口数相关，因此返回的是一个列表，在后续处理后用于剔除掉满足标识符部分的内容。
    """
    
    res = ["\n"]
    for i in range(nb_gNB_RX):
        for j in range(nb_UE_ports):
            res.append(f"====================== UE port {j} --> gNB Rx antenna {i} ======================\n")
    return res

def lowess(x:np.ndarray, y:np.ndarray, f:float=0.1, iter:int=3)->list:
    """lowess(x, y, f=2./3., iter=3) -> yest
    Lowess smoother: Robust locally weighted regression.
    The lowess function fits a nonparametric regression curve to a scatterplot.
    The arrays x and y contain an equal number of elements; each pair
    (x[i], y[i]) defines a data point in the scatterplot. The function returns
    the estimated (smooth) values of y.
    The smoothing span is given by f. A larger value for f will result in a
    smoother curve. The number of robustifying iterations is given by iter. The
    function will run faster with a smaller number of iterations.
    """
    n = len(x)
    r = int(ceil(f * n))
    h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
    w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)
    w = (1 - w ** 3) ** 3
    yest = np.zeros(n)
    delta = np.ones(n)
    for iteration in range(iter):
        for i in range(n):
            weights = delta * w[:, i]
            b = np.array([np.sum(weights * y), np.sum(weights * y * x)])
            A = np.array([[np.sum(weights), np.sum(weights * x)],
                          [np.sum(weights * x), np.sum(weights * x * x)]])
            beta = linalg.solve(A, b)
            yest[i] = beta[0] + beta[1] * x[i]

        residuals = y - yest
        s = np.median(np.abs(residuals))
        delta = np.clip(residuals / (6.0 * s), -1, 1)
        delta = (1 - delta ** 2) ** 2

    return yest.tolist()

def process_phase(raw_phase:list)->list:
    """
    Unwrap Phase data and perform linear transform. 
    Return the preprocessed_data in list.
    """
    
    m = [i for i in range(-272,272,2)]
    F = np.unwrap(np.array(raw_phase)).tolist()
    k_ = (F[-1]-F[0])/(m[-1]-m[0])
    b_ = sum(F)/len(F)
    return [F[i] - k_*m[i] - b_ for i in range(len(F))]

def extract_wireless_data(data:list,tags)->list:
    """
    返回去除标识符的无线数据。
    """
    for each in tags:
        data = list(filter(lambda x : x != each, data))
    SPLIT_SPACE = nb_gNB_RX * nb_UE_ports
    data = [data[i:i+SPLIT_SPACE] for i in range(0,len(data),SPLIT_SPACE)]
    return data

def extract_wireless_time_data(raw_data):
    '''
    提取无线数据的记录时间。
    '''
    return [each.split(',')[2][12:] for each in raw_data]

def fix_0_in_wireless_time(data):
    '''
    修补无线数据提取时小数点位数不足6位的情况，将其在高位补0.
    '''
    res = []
    for each in data:
        each_head, each_last = each.split('.')
        each_last = (6 - len(each_last)) * '0' + each_last    # 正常来说应该是6位小数，不足需要在前面添0
        each = f"{each_head}.{each_last}"
        res.append(float(each))
    return res
def find_nearest_element(a, b):
    '''
    根据b集合元素，在a集合中找到每个元素之间相差最小的元素的下标。
    '''
    result = []
    j = 0  # a的索引
    for num in b:
        while j < len(a) - 1 and abs(a[j + 1] - num) < abs(a[j] - num):
            j += 1
        result.append(j)
    return result

def is_in_time_range(valid_time, time_data):
    '''
    双指针，valid_time升序排列，time_data也升序排列。valid_time是一个二维列表，其中的每一个元素都是二元列表，代表了动作起止时间。
    time_data是从无线数据中提取的时间数据，用于判断其中的元素是否在valid_time的每一个二元列表的时间区间内，如果是的话，则代表
    在这个时间段内的此无线数据是动作数据，否则为静止状态的数据。

    该函数适用于SFV-wireless。返回传入的数据中在valid_time中的time_data中的下标。
    '''
    i, j = 0, 0
    result = []
    
    while i < len(valid_time) and j < len(time_data):
        if time_data[j] < valid_time[i][0]:
            # 浮点数小于valid_time[i]的起始时间，在时间段之前
            j += 1
        elif time_data[j] > valid_time[i][1]:
            # 浮点数大于valid_time[i]的结束时间，在时间段之后
            i += 1
        else:
            result.append(j)
            j += 1
    
    return result

def process_complex(data:dict)->list:
    """
    将去除标识符的无线数据整合并将其转换成复数。便于下一步的处理。
    """
    return [complex(each['re'],each['im']) for each in data]

def complex2argument(complex_matrix:list[complex])->list:
    """
    将一个sample内的复数转换成幅度和相角矩阵。一个sample是指从一根天线得到的若干个子载波组成的一维数据。
    返回两个与sample相同长度的一维矩阵。
    """
    amp = [abs(each) for each in complex_matrix]
    phase = [cmath.phase(each) for each in complex_matrix]
    return amp, phase

In [6]:
IS_SMOOTH = False
IS_PHASE = False

In [7]:
for index in range(1,7):
    wireless_data = rf"{wireless_data_path}\{index}"
    file_list = os.listdir(wireless_data)
    log_file_name = [file.split(".")[0][:-5] for file in file_list if file.endswith('.log')][0]

    label_file = rf"{time_data_path}\{index}\time.txt"
    data_file = rf"{wireless_data}\{log_file_name}-data.log"
    data_time = rf"{wireless_data}\{log_file_name}-time.log"
    print(f"[ Wireless ] processing file : {data_file} now !")
    with open (label_file) as f:
        labels = f.readlines()
    f.close()

    with open (data_file) as f:
        data = f.readlines()
    f.close()

    with open (data_time) as f:
        time_data = f.readlines()
    f.close()
    del f, label_file, data_file, data_time

    temp = extract_wireless_time_data(time_data)
    wireless_time = fix_0_in_wireless_time(temp)
    tags = generate_data_split_tags(nb_gNB_RX, nb_UE_ports)
    data = extract_wireless_data(data, tags)

    start_time = float(labels[0][2:])
    end_time = float(labels[-1][2:])
    wireless_start_idx, wireless_end_idx = find_nearest_element(wireless_time,[start_time,end_time])
    wireless_time = wireless_time[wireless_start_idx:wireless_end_idx]

    data = data[wireless_start_idx:wireless_end_idx]
    processed_data = [[process_complex(eval(complex_num)) 
                                        for complex_num in sample] 
                                        for sample in data[:]
                                        ]
    fall_tags= [[float(labels[i+2].split(":")[1]),float(labels[i+3].split(":")[1])] 
                        for i in range(len(labels)) if i % 4 == 0]
    stand_tags= [[float(labels[i].split(":")[1]),float(labels[i+1].split(":")[1])] 
                        for i in range(len(labels)) if i % 4 == 0]

    print(f"    [ Wireless ] Length of the file data is {len(data)}.")
    print(f"    [ Wireless ] Length of the file time is {len(wireless_time)}.")
    print(f"    [ LABEL ] Length of FALL LABEL tag is {len(fall_tags)}.")
    print(f"    [ LABEL ] Length of STAND LABEL tag is {len(stand_tags)}.")

    fall_result = is_in_time_range(fall_tags,wireless_time)
    stand_result = is_in_time_range(stand_tags,wireless_time)       

    IS_SMOOTH = False
    IS_PHASE = False

    # 处理跌倒状态
    fall_amp_all = []
    fall_phi_all = []

    stand_amp_all = []
    stand_phi_all = []
    for frame_id,frame in enumerate(processed_data):
        fall_amp_list = []
        fall_phase_list = []

        stand_amp_list = []
        stand_phase_list = []

        # 跌倒
        if frame_id in fall_result:
            for gNB_data in frame:
                amp, phase = complex2argument(gNB_data)
                if IS_SMOOTH:
                    amp = lowess(np.array([_ for _ in range(len(amp))]), np.array(amp))
                if IS_PHASE:
                    phase = process_phase(phase)
                fall_amp_list.append(amp)
                fall_phase_list.append(phase)
            fall_amp_all.append(fall_amp_list)
            fall_phi_all.append(fall_phase_list)
        elif frame_id in stand_result:
            for gNB_data in frame:
                amp, phase = complex2argument(gNB_data)
                if IS_SMOOTH:
                    amp = lowess(np.array([_ for _ in range(len(amp))]), np.array(amp))
                if IS_PHASE:
                    phase = process_phase(phase)
                stand_amp_list.append(amp)
                stand_phase_list.append(phase)
            stand_amp_all.append(stand_amp_list)
            stand_phi_all.append(stand_phase_list)

    fall_amp_all = np.array(fall_amp_all)
    fall_phi_all = np.array(fall_phi_all)
    fall_amp_save = np.reshape(np.transpose(fall_amp_all,(0,2,1)),(len(fall_amp_all),272,1,2))
    fall_phi_save = np.reshape(np.transpose(fall_phi_all,(0,2,1)),(len(fall_phi_all),272,1,2))

    stand_amp_all = np.array(stand_amp_all)
    stand_phi_all = np.array(stand_phi_all)
    stand_amp_save = np.reshape(np.transpose(stand_amp_all,(0,2,1)),(len(stand_amp_all),272,1,2))
    stand_phi_save = np.reshape(np.transpose(stand_phi_all,(0,2,1)),(len(stand_phi_all),272,1,2))

    # 保存文件
    fall_save_path = f"{output_raw_path}\\demo{index}\\fall\\"
    stand_save_path = f"{output_raw_path}\\demo{index}\\stand\\"
    print(f"[ FILE ] Files will be saved in '{fall_save_path}' .")
    print(f"[ FILE ] Files will be saved in '{stand_save_path}' .")
    fall_dir_path = os.path.dirname(fall_save_path)
    stand_dir_path = os.path.dirname(stand_save_path)
    # 如果目录不存在，则创建目录
    if not os.path.exists(fall_dir_path):
        os.makedirs(fall_dir_path)

    if not os.path.exists(stand_save_path):
        os.makedirs(stand_save_path)
    np.save(rf'{fall_save_path}\amp.npy',fall_amp_save)
    np.save(rf'{fall_save_path}\phi.npy',fall_phi_save)

    np.save(rf'{stand_save_path}\amp.npy',stand_amp_save)
    np.save(rf'{stand_save_path}\phi.npy',stand_phi_save)
    print("================================================================")



[ Wireless ] processing file : CLICKTIME\data\1\2024-05-15_14-55-49-data.log now !
    [ Wireless ] Length of the file data is 2781.
    [ Wireless ] Length of the file time is 2781.
    [ LABEL ] Length of FALL LABEL tag is 11.
    [ LABEL ] Length of STAND LABEL tag is 11.
[ FILE ] Files will be saved in 'FALL-set\demo1\fall\' .
[ FILE ] Files will be saved in 'FALL-set\demo1\stand\' .
[ Wireless ] processing file : CLICKTIME\data\2\2024-05-15_15-01-49-data.log now !
    [ Wireless ] Length of the file data is 3652.
    [ Wireless ] Length of the file time is 3652.
    [ LABEL ] Length of FALL LABEL tag is 15.
    [ LABEL ] Length of STAND LABEL tag is 15.
[ FILE ] Files will be saved in 'FALL-set\demo2\fall\' .
[ FILE ] Files will be saved in 'FALL-set\demo2\stand\' .
[ Wireless ] processing file : CLICKTIME\data\3\2024-05-15_15-27-14-data.log now !
    [ Wireless ] Length of the file data is 3404.
    [ Wireless ] Length of the file time is 3404.
    [ LABEL ] Length of FALL LABEL 

In [11]:
def process_phi_data(raw_data:np.ndarray):
    raw_data = raw_data.tolist()
    phi_all = []
    for each in raw_data:
        phi_1 = process_phase([each[i][0][0] for i in range(272)])
        phi_2 = process_phase([each[i][0][1] for i in range(272)])
        phi_data = [[[phi_1[i],phi_2[i]]] for i in range(272)]
        phi_all.append(phi_data)
    return np.array(phi_all)

In [9]:
final_save_root = "FINAL"

In [12]:
for person in range(1,7):
    for action in ['fall','stand']:
        print(f"    [ Data ] Processing {person} - {action} now ! >>>>>>>>>>>>")
        save_path = rf"{final_save_root}\demo{person}\{action}\\"
        dir_path = os.path.dirname(save_path)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path,exist_ok=True)
            
        amp_file = rf"{output_raw_path}\demo{person}\{action}\amp.npy"
        phi_file = rf"{output_raw_path}\demo{person}\{action}\phi.npy"
        amp_data = np.load(amp_file)
        phi_data = np.load(phi_file)

        el_amp = amp_data
        el_phase = process_phi_data(phi_data)
        all_data = np.concatenate([el_amp,el_phase],axis=2)
        all_data = all_data.transpose(0,1,3,2)
        np.save(rf"{dir_path}\all.npy", all_data, )
print("All Data Processing Task finished.")

    [ Data ] Processing 1 - fall now ! >>>>>>>>>>>>
    [ Data ] Processing 1 - stand now ! >>>>>>>>>>>>
    [ Data ] Processing 2 - fall now ! >>>>>>>>>>>>
    [ Data ] Processing 2 - stand now ! >>>>>>>>>>>>
    [ Data ] Processing 3 - fall now ! >>>>>>>>>>>>
    [ Data ] Processing 3 - stand now ! >>>>>>>>>>>>
    [ Data ] Processing 4 - fall now ! >>>>>>>>>>>>
    [ Data ] Processing 4 - stand now ! >>>>>>>>>>>>
    [ Data ] Processing 5 - fall now ! >>>>>>>>>>>>
    [ Data ] Processing 5 - stand now ! >>>>>>>>>>>>
    [ Data ] Processing 6 - fall now ! >>>>>>>>>>>>
    [ Data ] Processing 6 - stand now ! >>>>>>>>>>>>
All Data Processing Task finished.


In [ ]:
print(all_data[0])

[[[ 1.48822041e+02 -1.07168403e-01]
  [ 1.04235311e+02 -1.35285046e-01]]

 [[ 1.56818366e+02 -8.02841669e-02]
  [ 7.88923317e+01 -3.45391687e-01]]

 [[ 1.38654246e+02 -6.75294289e-02]
  [ 9.97246208e+01  1.39180483e-01]]

 ...

 [[ 1.06042444e+02 -4.43743651e-02]
  [ 2.00731163e+02  3.18788481e-03]]

 [[ 8.45990544e+01 -7.11968070e-02]
  [ 2.07790760e+02  8.75540206e-03]]

 [[ 1.38003623e+02 -1.07168403e-01]
  [ 2.03184645e+02 -1.35285046e-01]]]
